In [3]:
!pip install --upgrade fake_useragent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=b7f01c7e26b995df0098296756b523e80ecd8e2e771f6290eb9d5e1454a77353
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
Successfully built fake-useragent


In [4]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 1.4 MB/s 
     |████████████████████████████████| 8.2 MB 3.6 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=f9fc02c9b921a7841b8e4c1fc91d30a8a191db96bb5d392aa14f2c86e2015b6c
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


# 1. Parsing

In [5]:
from fake_useragent import UserAgent
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import re

In [6]:
session = requests.session()

In [7]:
ua = UserAgent(verify_ssl=False)

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 154, in load
    for item in get_browsers(verify_ssl=verify_ssl):
  File "/usr/local/lib/python3.7/dist-packages/fake_useragent/utils.py", line 99, in get_browsers
    html = html.split('<table class="w3-table-all notranslate">')[1]
IndexError: list index out of range


Собираю названия популярных в последнеее время фильмов.

In [8]:
req1 = session.get('https://www.metacritic.com/browse/movies/release-date/theaters/userscore', headers={'User-Agent': ua.random})
page_all = req1.text
soup_all = BeautifulSoup(page_all, 'html.parser')

In [9]:
titles = []
for link in soup_all.find_all('a', {'class':'title'}):
    titles.append(link.get('href'))

In [10]:
titles = titles[:20]
print(titles)

['/movie/dragon-ball-super-super-hero', '/movie/take-the-night', '/movie/petrovs-flu', '/movie/block-party-2022', '/movie/both-sides-of-the-blade', '/movie/marcel-the-shell-with-shoes-on', '/movie/moonage-daydream', '/movie/official-competition', '/movie/thirteen-lives', '/movie/top-gun-maverick', '/movie/the-sea-beast', '/movie/bullet-train', '/movie/emily-the-criminal', '/movie/dinner-in-america', '/movie/a-jazzmans-blues', '/movie/barbarian', '/movie/benediction', '/movie/hustle-2022', '/movie/speak-no-evil', '/movie/fire-of-love']


Для каждого фильма собираю положительные и отрицательные отзывы.

In [11]:
def at_all(film_name, fin_pos, fin_neg):
  req = session.get(f'https://www.metacritic.com{film_name}/user-reviews', headers={'User-Agent': ua.random})
  page = req.text
  soup = BeautifulSoup(page, 'html.parser')
  reviews_body = soup.find_all('div', {'class':'review pad_top1'})

  def parser(review_piece, list1):
    if '"blurb blurb_expanded"' in str(review_piece):
      review = i.find('span', {'class':'blurb blurb_expanded'}).text
    else:
      review = i.find('div', {'class':'review_body'}).text
    list1.append(review)
    return (list1)

  positives = []
  negatives = []
  for i in reviews_body:
    if '"metascore_w user large movie negative indiv"' in str(i):
      negatives = parser(i, negatives)
    if '"metascore_w user large movie positive indiv"' in str(i):
      positives = parser(i, positives)

  fin_pos.append(positives)
  fin_neg.append(negatives)

  return fin_pos, fin_neg

In [12]:
final_pos = []
final_neg = []
for i in titles:
  final_pos = at_all(i, final_pos, final_neg)[0]
  final_neg = at_all(i, final_pos, final_neg)[1]

In [13]:
final_neg = final_neg[:-1:2]
final_pos = final_pos[:-1:2]

In [14]:
def listmerge(lstlst):
  all=[]
  for lst in lstlst:
    for el in lst:
      all.append(el.strip('\n').strip('\xa0'))
  return all

In [15]:
overal_neg = listmerge(final_neg)[:45]
overal_pos = listmerge(final_pos)[:45]

Делю на тренировочную и тестовую выборки.

In [16]:
test_neg = overal_neg[0:5]
test_pos = overal_pos[0:5]
train_neg = overal_neg[5:-1]
train_pos = overal_pos[5:-1]

# 2. Tokens and lemmas


In [17]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('punkt')
from collections import Counter

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [18]:
from nltk.corpus import stopwords

Токенизирую и попутно чищу от стоп-слов. Так как встречались отзывы не только на английском языке, очистила и от французских стоп-слов.

In [19]:
def tokenizer(reviews):
  sw = stopwords.words(['english', 'french'])
  tokenized = []
  for i in reviews:
    words = [w.lower() for w in word_tokenize(i) if w.isalpha()]
    filtered = [w for w in words if w not in sw]
    for j in filtered:
      tokenized.append(j)
  return tokenized

In [20]:
neg_tok = tokenizer(train_neg)
pos_tok = tokenizer(train_pos)

In [21]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Леммматизирую.

In [22]:
def lemmatizer(words):
  lemmas = []
  for word in words:
      lemmas.append(morph.parse(word)[0].normal_form)
  return lemmas

In [23]:
neg_lem = lemmatizer(neg_tok)
pos_lem = lemmatizer(pos_tok)

Составляю частотный словарь, чтобы отфильтровать низкочастотные слова.

In [24]:
def freq_dict(list_of_words):
  cnt = Counter()
  for word in list_of_words:
    if word.isalpha():
      cnt[word] += 1
  return dict(cnt)

In [25]:
neg_freqs, pos_freqs = freq_dict(neg_lem), freq_dict(pos_lem)

In [26]:
neg_freqs = {key:val for key, val in neg_freqs.items() if val >= 3}
pos_freqs = {key:val for key, val in pos_freqs.items() if val >= 3}

# 3. Unique words


При помощи операции с множествами составляю два множества уникальных слов для каждого типа отзывов.

In [27]:
neg_words, pos_words = set(neg_freqs.keys()), set(pos_freqs.keys())

In [28]:
unique_neg = neg_words.difference(pos_words)
unique_pos = pos_words.difference(neg_words)

# 4. Sentiment analysis

Составляю словарь для сравнения с результатом работы функции.

In [29]:
checklist = []
for i in test_neg:
  to_check = {}
  to_check['review'] = i
  to_check['sense'] = 0
  checklist.append(to_check)
for i in test_pos:
  to_check ={}
  to_check['review'] = i
  to_check['sense'] = 1
  checklist.append(to_check)

Функция подсчитывает, каких слов в тексте отзыва больше и в соответствии с этим определяет тип отзыва. Если "отрицательных" и "положительных" слов одинаковое количество, тексту присвается нейтральный статус (2).

In [30]:
def sentimentor(review):
  negs = 0
  poss = 0
  so = {}
  for word in nltk.word_tokenize(review.lower()):
    if morph.parse(word)[0].normal_form in unique_neg:
      negs +=1
    elif morph.parse(word)[0].normal_form in unique_pos:
      poss +=1
  so['review'] = review
  so['inneg'] = negs
  so['inpos'] = poss
  if negs > poss:
    so['sense'] = 0
  elif negs < poss:
    so['sense'] = 1
  else:
    so['sense'] = 2
  return so

In [31]:
whole_test = test_neg + test_pos

In [32]:
all = []
for i in whole_test:
  all.append(sentimentor(i))

In [33]:
def get_values(listik):
  items = []
  for i in listik:
    items.append(i['sense'])
  return items

Подсчитываю accuracy с помощью sklearn. Передаю туда рещультаты работы функции и "парвильные" ответы.

In [34]:
import sklearn

In [35]:
accuracy = sklearn.metrics.accuracy_score(get_values(checklist), get_values(all))
accuracy

0.7

# 5. Another solution


1. Один из способов - считать эмбеддинги для каждого слова. Эмбеддинги "отрицательных" слов скорее всего будут похожи между собой и сильно отличаться от "положительных". Так можно посчитать значение (что-то типа среднего) для каждого текста --> получить эмбеддинг для текста отзыва целиком. Можно посмотреть несколько таких эмбеддингов и сделать предположение вроде "у отрицательных отзывов эмбеддинги отрицательные, а у положительных - положительные".  
2. Можно улучшить тщательнее подсчитывая слова. Например, можно выделять ключевые слова (есть разные модели для keywords extraction), отфильтровывать их в зависимости от тематики отзывов в целом(вероятно, вручную) и использовать модель spacy, которая выделяет зависимости (синатксис). Необходимо брать только зависимые от ключевых слов для каждого отзыва. Тогда отзывы о фильмах по типу (утрированный пример): "Я недавно купил хорошую машину, а фильм, кстати, ужасный" будет отнесен к отрицательным, так как "хороший" о машине здесь нерелевантен. (Действуя напрямую, как сделано выше, этот отзыв посчитается за нейтральный.)